In [1]:
pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.28.1 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.

#Original case (11 nodes)

In [3]:
"""Capacited Vehicles Routing Problem (CVRP)."""
import psutil
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

print('RAM usage:', psutil.Process().memory_info().rss / 1024 ** 2, 'MB')


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
      [0,808,654,275,134,357,545,236,123,484,645],
      [808,0,156,724,752,860,960,870,849,1036,1207],
      [654,156,0,571,596,710,854,731,701,889,1063],
      [275,724,571,0,141,151,796,503,396,331,511],
      [134,752,596,141,0,242,663,364,255,396,570],
      [357,860,710,151,242,0,900,593,478,181,360],
      [545,960,854,796,663,900,0,313,429,1023,1171],
      [236,870,731,503,364,593,313,0,116,710,859],
      [123,849,701,396,255,478,429,116,0,595,746],
      [484,1036,889,331,396,181,1023,710,595,0,179],
      [645,1207,1063,511,570,360,1171,859,746,179,0],
        # fmt: on
    ]
    data["demands"] = [0,40,56,64,11,92,38,53,64,5,75]
    data["vehicle_capacities"] = [100,100,100,100,100,100]
    data["num_vehicles"] = 6
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"SOLUTION FOUND...\n\n\n")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Remaining({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Remaining({route_load})\n"
        plan_output += f"Time of the route: {route_distance/100} minutes\n"
        plan_output += f"Load of the route: {route_load/100}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Objective: {solution.ObjectiveValue()/100+total_load/100*30} minutes in total.")
    print(f"Total time of all routes: {total_distance/100} minutes")
    print(f"Total load of all routes: {total_load/100}")
    print(f"Total time spent loading and unloading: {total_load/100*30} minutes")



def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    #Se utiliza en primera instancia una heurística , que comienza por un nodo inicial que por consiguiente busca otro nodo , haciendo una ruta de la manera más económica o más barata, y luego
    #extiende a otro nodo , extendiendo la ruta con la misma lógica
    #Asi mismo se utiliza una metaheurística para mejorar la solución inicial , usamos guided local search es un proceso iterativo que busca mejores soluciones  a  la inicial y se continua el proceso hasta no mejorar la situación actual
    #Es parecio a que local search  a diferencia es que en guided local search para poder salir de óptimos locales , se agregan penalizaciones, lo que lo hace más efectivo.
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

RAM usage: 123.8515625 MB
SOLUTION FOUND...



Route for vehicle 0:
 0 Remaining(0) ->  2 Remaining(56) ->  1 Remaining(96) ->  0 Remaining(96)
Time of the route: 16.18 minutes
Load of the route: 0.96

Route for vehicle 1:
 0 Remaining(0) ->  9 Remaining(5) ->  10 Remaining(80) ->  0 Remaining(80)
Time of the route: 13.08 minutes
Load of the route: 0.8

Route for vehicle 2:
 0 Remaining(0) ->  5 Remaining(92) ->  0 Remaining(92)
Time of the route: 7.14 minutes
Load of the route: 0.92

Route for vehicle 3:
 0 Remaining(0) ->  4 Remaining(11) ->  3 Remaining(75) ->  0 Remaining(75)
Time of the route: 5.5 minutes
Load of the route: 0.75

Route for vehicle 4:
 0 Remaining(0) ->  7 Remaining(53) ->  6 Remaining(91) ->  0 Remaining(91)
Time of the route: 10.94 minutes
Load of the route: 0.91

Route for vehicle 5:
 0 Remaining(0) ->  8 Remaining(64) ->  0 Remaining(64)
Time of the route: 2.46 minutes
Load of the route: 0.64

Objective: 204.7 minutes in total.
Total time of all routes: 55.3 mi

#Medium size case (16 nodes)

In [3]:
%%time
import psutil
# Your code here
print('RAM usage:', psutil.Process().memory_info().rss / 1024 ** 2, 'MB')
"""Capacited Vehicles Routing Problem (CVRP)."""

"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
       [0,592,759,409,677,999,470,872,796,714,886,951,487,574,937,472],
       [592,0,848,499,577,547,823,688,665,944,943,644,551,910,859,583],
       [759,848,0,453,950,888,673,735,788,442,842,943,657,721,457,691],
       [409,499,453,0,798,965,484,603,724,447,531,580,543,627,773,960],
       [677,577,950,798,0,597,656,692,787,954,768,601,783,912,770,955],
       [999,547,888,965,597,0,523,994,548,609,962,811,441,458,436,486],
       [470,823,673,484,656,523,0,675,574,954,771,584,844,888,989,686],
       [872,688,735,603,692,994,675,0,810,850,648,580,723,660,539,998],
       [796,665,788,724,787,548,574,810,0,995,628,907,521,726,869,687],
       [714,944,442,447,954,609,954,850,995,0,749,529,616,781,424,467],
       [886,943,842,531,768,962,771,648,628,749,0,707,574,548,941,979],
       [951,644,943,580,601,811,584,580,907,529,707,0,488,433,632,436],
       [487,551,657,543,783,441,844,723,521,616,574,488,0,747,568,576],
       [574,910,721,627,912,458,888,660,726,781,548,433,747,0,956,722],
       [937,859,457,773,770,436,989,539,869,424,941,632,568,956,0,729],
       [472,583,691,960,955,486,686,998,687,467,979,436,576,722,729,0],
        # fmt: on
    ]
    data["demands"] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8]
    data["vehicle_capacities"] = [15, 15, 15, 15]
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Solution time: {solution.ObjectiveValue()} minutes.")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Remaining({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Remaining({route_load})\n"
        plan_output += f"Time of the route: {route_distance/100} minutes\n"
        plan_output += f"Load of the route: {route_load/100}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

RAM usage: 117.12109375 MB
Solution time: 10032 minutes.
Route for vehicle 0:
 0 Remaining(0) ->  12 Remaining(2) ->  11 Remaining(3) ->  7 Remaining(11) ->  4 Remaining(15) ->  0 Remaining(15)
Time of the route: 29.24 minutes
Load of the route: 0.15

Route for vehicle 1:
 0 Remaining(0) ->  15 Remaining(8) ->  0 Remaining(8)
Time of the route: 9.44 minutes
Load of the route: 0.08

Route for vehicle 2:
 0 Remaining(0) ->  1 Remaining(1) ->  8 Remaining(9) ->  10 Remaining(11) ->  13 Remaining(15) ->  0 Remaining(15)
Time of the route: 30.07 minutes
Load of the route: 0.15

Route for vehicle 3:
 0 Remaining(0) ->  6 Remaining(4) ->  5 Remaining(6) ->  14 Remaining(10) ->  9 Remaining(11) ->  2 Remaining(12) ->  3 Remaining(14) ->  0 Remaining(14)
Time of the route: 31.57 minutes
Load of the route: 0.14

CPU times: user 996 ms, sys: 2.96 ms, total: 999 ms
Wall time: 1.01 s


#Large size case (21 nodes)

In [5]:
%%time
import psutil
# Your code here
print('RAM usage:', psutil.Process().memory_info().rss / 1024 ** 2, 'MB')
"""Capacited Vehicles Routing Problem (CVRP)."""

"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
        [0,487,413,714,993,520,681,826,870,684,676,724,934,627,995,973,917,656,536,607,863],
        [487,0,823,748,682,986,833,619,773,690,963,837,908,849,726,890,453,494,459,736,563],
        [413,823,0,700,950,413,934,904,609,855,868,873,972,800,456,680,404,705,743,942,966],
        [714,748,700,0,811,429,771,902,407,537,776,548,659,526,977,932,724,512,637,760,955],
        [993,682,950,811,0,739,674,769,626,625,628,769,528,424,692,755,534,403,626,844,817],
        [520,986,413,429,739,0,597,472,646,535,613,875,723,502,727,995,623,656,747,507,919],
        [681,833,934,771,674,597,0,489,474,899,743,496,682,432,515,854,828,573,532,841,493],
        [826,619,904,902,769,472,489,0,898,605,814,765,819,985,618,938,774,422,860,754,451],
        [870,773,609,407,626,646,474,898,0,569,566,709,994,791,620,514,734,619,646,500,815],
        [684,690,855,537,625,535,899,605,569,0,721,848,690,403,638,571,982,779,509,729,500],
        [676,963,868,776,628,613,743,814,566,721,0,800,553,519,785,789,796,746,986,478,984],
        [724,837,873,548,769,875,496,765,709,848,800,0,417,847,750,725,824,564,970,577,839],
        [934,908,972,659,528,723,682,819,994,690,553,417,0,964,976,732,985,546,448,476,693],
        [627,849,800,526,424,502,432,985,791,403,519,847,964,0,932,608,442,898,428,810,580],
        [995,726,456,977,692,727,515,618,620,638,785,750,976,932,0,706,744,610,984,674,809],
        [973,890,680,932,755,995,854,938,514,571,789,725,732,608,706,0,625,518,945,551,438],
        [917,453,404,724,534,623,828,774,734,982,796,824,985,442,744,625,0,732,959,988,520],
        [656,494,705,512,403,656,573,422,619,779,746,564,546,898,610,518,732,0,608,688,462],
        [536,459,743,637,626,747,532,860,646,509,986,970,448,428,984,945,959,608,0,973,723],
        [607,736,942,760,844,507,841,754,500,729,478,577,476,810,674,551,988,688,973,0,555],
        [863,563,966,955,817,919,493,451,815,500,984,839,693,580,809,438,520,462,723,555,0],
        # fmt: on
    ]

    data["demands"] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8,7,6,5,8,8,7,6,5,8]
    data["vehicle_capacities"] = [15, 15, 15, 15,15,15,15,15,15,15]
    data["num_vehicles"] = 10
    data["depot"] = 0
    return data

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

RAM usage: 124.87109375 MB
Solution time: 14513 minutes.
Route for vehicle 0:
 0 Remaining(0) ->  17 Remaining(7) ->  20 Remaining(15) ->  0 Remaining(15)
Time of the route: 19.81 minutes
Load of the route: 0.15

Route for vehicle 1:
 0 Remaining(0) ->  10 Remaining(2) ->  4 Remaining(6) ->  12 Remaining(8) ->  18 Remaining(14) ->  0 Remaining(14)
Time of the route: 28.16 minutes
Load of the route: 0.14

Route for vehicle 2:
 0 Remaining(0) ->  2 Remaining(1) ->  14 Remaining(5) ->  7 Remaining(13) ->  5 Remaining(15) ->  0 Remaining(15)
Time of the route: 24.79 minutes
Load of the route: 0.15

Route for vehicle 3:
 0 Remaining(0) ->  11 Remaining(1) ->  6 Remaining(5) ->  8 Remaining(13) ->  3 Remaining(15) ->  0 Remaining(15)
Time of the route: 28.15 minutes
Load of the route: 0.15

Route for vehicle 4:
 0 Remaining(0) ->  0 Remaining(0)
Time of the route: 0.0 minutes
Load of the route: 0.0

Route for vehicle 5:
 0 Remaining(0) ->  1 Remaining(1) ->  16 Remaining(9) ->  13 Remaining(

#Maximum size case (26 nodes)

In [4]:
%%time
import psutil
# Your code here
print('RAM usage:', psutil.Process().memory_info().rss / 1024 ** 2, 'MB')
"""Capacited Vehicles Routing Problem (CVRP)."""

"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
        [0,592,759,409,677,999,470,872,796,714,886,951,487,574,937,472,411,642,803,199,525,960,600,338,510,222],
        [592,0,848,499,577,547,823,688,665,944,943,644,551,910,859,583,125,697,766,248,745,917,535,601,199,483],
        [759,848,0,453,950,888,673,735,788,442,842,943,657,721,457,691,968,592,330,566,758,103,311,800,706,421],
        [409,499,453,0,798,965,484,603,724,447,531,580,543,627,773,960,305,186,480,899,342,284,429,206,298,112],
        [677,577,950,798,0,597,656,692,787,954,768,601,783,912,770,955,334,696,283,937,653,679,719,187,552,881],
        [999,547,888,965,597,0,523,994,548,609,962,811,441,458,436,486,283,771,670,352,894,489,512,827,756,630],
        [470,823,673,484,656,523,0,675,574,954,771,584,844,888,989,686,582,715,429,667,218,703,525,663,140,241],
        [872,688,735,603,692,994,675,0,810,850,648,580,723,660,539,998,477,863,417,931,624,818,396,817,688,731],
        [796,665,788,724,787,548,574,810,0,995,628,907,521,726,869,687,279,879,778,431,947,587,688,747,562,177],
        [714,944,442,447,954,609,954,850,995,0,749,529,616,781,424,467,788,123,213,601,670,745,655,545,887,501],
        [886,943,842,531,768,962,771,648,628,749,0,707,574,548,941,979,825,742,534,882,416,614,210,471,876,235],
        [951,644,943,580,601,811,584,580,907,529,707,0,488,433,632,436,275,715,177,793,298,865,286,342,664,409],
        [487,551,657,543,783,441,844,723,521,616,574,488,0,747,568,576,644,842,773,637,418,999,568,999,244,509],
        [574,910,721,627,912,458,888,660,726,781,548,433,747,0,956,722,201,444,888,689,236,331,518,299,823,640],
        [937,859,457,773,770,436,989,539,869,424,941,632,568,956,0,729,542,775,873,450,699,732,185,719,365,467],
        [472,583,691,960,955,486,686,998,687,467,979,436,576,722,729,0,303,825,999,699,164,400,468,696,795,703],
        [411,125,968,305,334,283,582,477,279,788,825,275,644,201,542,303,0,256,658,144,477,501,282,677,336,675],
        [642,697,592,186,696,771,715,863,879,123,742,715,842,444,775,825,256,0,724,857,392,749,470,615,412,489],
        [803,766,330,480,283,670,429,417,778,213,534,177,773,888,873,999,658,724,0,683,218,573,654,710,784,287],
        [199,248,566,899,937,352,667,931,431,601,882,793,637,689,450,699,144,857,683,0,325,914,534,444,357,599],
        [525,745,758,342,653,894,218,624,947,670,416,298,418,236,699,164,477,392,218,325,0,702,176,373,697,490],
        [960,917,103,284,679,489,703,818,587,745,614,865,999,331,732,400,501,749,573,914,702,0,220,283,421,553],
        [600,535,311,429,719,512,525,396,688,655,210,286,568,518,185,468,282,470,654,534,176,220,0,159,319,267],
        [338,601,800,206,187,827,663,817,747,545,471,342,999,299,719,696,677,615,710,444,373,283,159,0,246,690],
        [510,199,706,298,552,756,140,688,562,887,876,664,244,823,365,795,336,412,784,357,697,421,319,246,0,126],
        [222,483,421,112,881,630,241,731,177,501,235,409,509,640,467,703,675,489,287,599,490,553,267,690,126,0],
        # fmt: on
    ]

    data["demands"] = [ 0, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8]
    data["vehicle_capacities"] = [15,15,15,15,15,15,15,15]
    data["num_vehicles"] = 8
    data["depot"] = 0
    return data

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.

    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

RAM usage: 119.30078125 MB
Solution time: 12216 minutes.
Route for vehicle 0:
 0 Remaining(0) ->  15 Remaining(2) ->  20 Remaining(4) ->  10 Remaining(12) ->  22 Remaining(14) ->  0 Remaining(14)
Time of the route: 18.62 minutes
Load of the route: 0.14

Route for vehicle 1:
 0 Remaining(0) ->  23 Remaining(4) ->  4 Remaining(5) ->  18 Remaining(13) ->  11 Remaining(14) ->  12 Remaining(15) ->  0 Remaining(15)
Time of the route: 19.6 minutes
Load of the route: 0.15

Route for vehicle 2:
 0 Remaining(0) ->  16 Remaining(4) ->  13 Remaining(6) ->  21 Remaining(7) ->  2 Remaining(15) ->  0 Remaining(15)
Time of the route: 18.05 minutes
Load of the route: 0.15

Route for vehicle 3:
 0 Remaining(0) ->  1 Remaining(4) ->  7 Remaining(6) ->  14 Remaining(10) ->  24 Remaining(14) ->  6 Remaining(15) ->  0 Remaining(15)
Time of the route: 27.94 minutes
Load of the route: 0.15

Route for vehicle 4:
 0 Remaining(0) ->  9 Remaining(4) ->  17 Remaining(12) ->  0 Remaining(12)
Time of the route: 14.7